## Fitting psychometric functions with more advanced statistics packages



In [3]:
from djchurchland.schema import Session, Task # import all schemas
import pandas as pd
import numpy as np

ses = pd.DataFrame((Session*Task.TrialSet() & 'subject_name = "JC098"' & 'session_datetime = "2023-02-15 15:12:48"').fetch());


In [4]:
# run the third cell before running this or import from btss
# rom btss.task_utils import fit_psychometric

x = ses.iloc[0].stim_values        # stim amplitudes
y = ses.iloc[0].response_values    # responses to the left 1, right -1 or no response 0
x = x[(np.isfinite(y)) & (y!= 0)]  # discard trials with no response
y = y[(np.isfinite(y)) & (y!= 0)]  # responses to the left (or right)


res = fit_psychometric(x,y==1)
%matplotlib qt
import pylab as plt
plt.plot(res['stims'],res['p_side'],'ko')
for s,c in zip(res['stims'],res['p_side_ci']):
    plt.plot(s*np.array([1,1]),c,'k-_')
nx = np.linspace(-20,20,100)
plt.plot(nx,res['function'](*res['fit_params'],nx))
res['fit'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          PsychometricRegression Results                          
==================================================================================
Dep. Variable:                          y   Log-Likelihood:                -324.26
Model:             PsychometricRegression   AIC:                             656.5
Method:                Maximum Likelihood   BIC:                             673.8
Date:                    Mon, 06 Mar 2023                                         
Time:                            14:30:49                                         
No. Observations:                     558                                         
Df Residuals:                         554                                         
Df Model:                               0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
bias           0.3907      6.243      0.063      0.950     -11.844      12.626
slope         27.5002      8.713      3.156      0.002      10.424      44.576
gamma1              0      0.154          0      1.000      -0.301       0.301
gamma2              0      0.181          0      1.000      -0.354       0.354
==============================================================================
"""

In [1]:
# the following is from github.com/jcouto/btss
from statsmodels.stats.proportion import proportion_confint
from scipy.special import erfc # import the complementary error function
from scipy.special import erf # import the error function
# This has example ways to fit the psychometric function and getting confidence intervals
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import minimize
import numpy as np

# weibull fit
def weibull(bias, slope, gamma1, gamma2, X):
    ''' weibull function with lapse rates
    '''
    return gamma1 + (1. - gamma1 - gamma2) * (erf((X - bias) / slope) + 1.) / 2.

# log likelihood when using minimize
def neg_log_likelihood_error(func, parameters, X, Y):
    '''
    Compute the log likelihood

    'func' is the (psychometric) function 
    'parameters' are the input parameters to 'func'
    'Y' is the binary response (correct = 1; incorrect=0)
    Joao Couto - Jan 2022    
    '''

    pX = func(*parameters, X)*0.99 + 0.005  # the predicted performance for X from the PMF
    # epsilon to prevent error in log(0)
    val = np.nansum(Y*np.log(pX) + (1-Y)*np.log(1-pX))
    return -1*val

def compute_proportions(stim_values, response_values):
    '''
    Computes the proportion of responses to each stimulus value.
Returns: 
    - stims    - unique stimulus intensities
    - p_side   - proportion of trials to the side
    - ci_side  - confidance intervals from binomial distribution with the wilson method
    - n_obs    - number of observations (trials for each)
    - n_side   - number of trials to the specific side


Joao Couto - Jan 2023
    '''
    
    stims = np.unique(stim_values)
    p_side = np.zeros_like(stims,dtype=float) 
    ci_side = np.zeros((len(stims),2),dtype=float)
    n_obs = np.zeros_like(stims,dtype=float)
    n_side = np.zeros_like(stims,dtype=float)
    for i,intensity in enumerate(stims):
        # number of times the subject licked to one of the sides 
        cnt = np.sum(response_values[stim_values == intensity]) 
        nobs = np.sum(stim_values == intensity) # number of observations (ntrials)
        n_obs[i] = nobs
        n_side[i] = cnt
        p_side[i] = cnt/nobs
        ci_side[i] = proportion_confint(cnt,nobs,method='wilson') # 95% confidence interval
    return stims,p_side,ci_side,n_obs,n_side

# statsmodels gives confidence and p values for the fit
class PsychometricRegression(GenericLikelihoodModel):
    def __init__(self, endog, exog, func = None, bounds = None,startpar_function = None,parnames = None, **kwds):
        '''
        Fits a psychometric with constraints function (constrained weibull e.g.)
        
        This is part of the tools for github.com/jcouto/btss
        
        Inputs:
            endog - are the response values for each trial
            exog  - are the stim intensities for each trial
            func  - the function to fit; default weibull with lapses
            bound - the constraints to the fit; default [(min(x),max(x)),(0.01,1000),(0,1),(0,1)]
            startpar_function - a function to get the start guess for the fit
            parnames - the names of the fit parameters
            
        Usage:
        
        ft = PsychometricRegression(response_values.astype(float),
                                    exog = stim_values.astype(float))
        res = ft.fit(min_required_stim_values = min_required_stim_values, full_output=True)
        print(res.summary())
            
        Joao Couto - Feb 2023
        '''

        super(PsychometricRegression, self).__init__(endog, exog, **kwds)
        if not func is None:
            self.fit_function = func
            self.bounds = None
            self.get_start_params = startpar_function
            if not parnames is None:
                self.exog_names[:] = parnames
        else:
            self.fit_function = weibull
            self.exog_names[:] = ['bias','slope','gamma1','gamma2']
            self.bounds = [(np.min(self.exog[:,0]),np.max(self.exog[:,0])),
                           (0.01,1000),
                           (0,1),(0,1)]
            self.get_start_params  = lambda x,y: [0,np.max(x)/2,y[0],1-y[-1]]
            
    
    def loglikeobs(self,params):
        pX = self.fit_function(*params, self.exog[:,0])
        pX[pX==0] = 0.001
        pX[pX==1] = 0.999
        # the predicted performance for X from the PMF
        val = np.nansum(self.endog*np.log(pX) + (1-self.endog)*np.log(1-pX))
        return val
    
    def fit(self, start_params=None,
            maxiter=100000,
            maxfun=10000,
            min_required_stim_values=6, **kwds):

        self.stims, self.p_side, self.ci_side, self.n_obs,self.n_side = compute_proportions(self.exog[:,0], self.endog)
        if len(self.stims) < min_required_stim_values:
            return None
        
        if start_params == None:
            # Reasonable starting values
            if not self.get_start_params is None:
                start_params = self.get_start_params(self.stims, self.p_side)
        if start_params is None:
            raise(ValueError("Need to provide start parameters or a function to compute them."))
        self.df_null = 0
        self.k_constant = len(start_params)
        self.df_resid = len(self.endog)-len(start_params)
        return super(PsychometricRegression, self).fit(
            start_params=start_params,
            maxiter=maxiter,
            maxfun=maxfun,
            method = 'lbfgs',
            bounds = self.bounds,
            disp = False,
            **kwds)
        
# main function to fit and compute statistics
def fit_psychometric(stim_values, response_values,
                     func = weibull,    # to fit the psychometric
                     min_required_stim_values = 6,  # min values required to fit the function
                     method = 'PsychometricRegression'):#'Nelder-Mead'):#'L-BFGS-B'):
    '''
    Fits a psychometric curve and computes points
    
    Joao Couto - Jan 2023
    '''
    if method == 'PsychometricRegression':
        # use PsychometricRegression (default weibull)
        ft = PsychometricRegression(response_values.astype(float),
                                    exog = stim_values.astype(float))
        res = ft.fit(min_required_stim_values = min_required_stim_values, full_output=True)
        if res is None:
            fit_res = None
            params = None
        else:
            fit_res = res
            params = res.params
        return dict(stims = ft.stims,
                    p_side = ft.p_side,
                    p_side_ci = ft.ci_side,
                    n_side = ft.n_side,
                    n_obs = ft.n_obs,
                    fit_params = params,
                    fit = fit_res,
                    function = ft.fit_function)
    
    stims,p_side,ci_side,n_obs,n_side = compute_proportions(stim_values, response_values)
    fit_res = None
    params = None
    if len(stims) >= min_required_stim_values and not func is None:

        opt_func = lambda pars: neg_log_likelihood_error(func, pars, 
                                                     stim_values,
                                                     response_values)
        # x0 is the initial guess for the fit, it is an important parameter
        x0 = [0.,0.1,p_side[0],1 - p_side[-1]]
        bounds = [(stims[0],stims[-1]),(0.0001,10),(0,0.7),(0,.7)]
        #import warnings
        #warnings.filterwarnings('ignore')
        options = dict(maxiter = 500*len(x0))
        if 'Neder' in method:
            options['adaptive'] = True
        fit_res = minimize(opt_func, x0,
                           options = options,
                           bounds = bounds, method = method)
        params = fit_res.x
    return dict(stims = stims,
                p_side = p_side,
                p_side_ci = ci_side,
                n_side = n_side,
                n_obs = n_obs,
                fit_params = params,
                fit = fit_res,
                function = func)
